In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.utils.data as data_utils

In [2]:
train_path = '/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/new/'

In [3]:
#input_ = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_input.npy')

In [4]:
#output_ = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_output.npy')

In [5]:
input_dev = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_input_dev.npy')

In [6]:
output_dev = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_output_dev.npy')

In [7]:
list_of_train_files = []
list_of_label_files = []
for path, subdirs, files in os.walk(train_path):
    for name in files:
        if name.startswith("ffnn_train"):
            list_of_train_files.append(os.path.join(path, name))
        if name.startswith("ffnn_labels"):
            list_of_label_files.append(os.path.join(path, name))
list_of_train_files = sorted(list_of_train_files)
list_of_label_files = sorted(list_of_label_files)
        

In [8]:
len(list_of_train_files)

100

In [9]:
len(list_of_label_files)

100

In [10]:
inp = np.load(list_of_train_files[0])

In [11]:
inp[0].shape

(1237, 1)

In [12]:
input_dev[0].shape

(1237, 1)

In [12]:
INPUT_DIM = 1237
HIDDEN_DIM1 = 1000
HIDDEN_DIM2 = 500
HIDDEN_DIM3 = 500
OUTPUT_DIM = 2
BATCH_SIZE = 32
NUM_EPOCHS = 10

In [19]:
def dataLoader(start_index, batch_size, inputs, labels, input_dim):
    '''
    This function provides the data to the network in batches from the starting position start_index.
    Args: 
       start_index (int): Position to start the batch from the dataset.
       batch_size (int): Batch size.
       inputs: The inputs to the model.
       labels: The expected outputs from the model.
    
    Returns: 
        train_tensor(torch tensor): Tensor of batch size.
        label_tensor(torch tensor): Tensor of batch size.
    '''
    
    inputs_tensor_list = []
    labels_tensor_list = []
    
    if len(inputs[i:]) < batch_size:
        batch_size = len(inputs[i:])
    
    for pos in range(start_index, start_index + batch_size):
        inputs_tensor_list.append((torch.from_numpy(inputs[pos].reshape(input_dim))).float())
        labels_tensor_list.append((torch.from_numpy(labels[pos])))
        
    inputs_tensor = torch.Tensor(batch_size, input_dim)
    labels_tensor = torch.LongTensor(batch_size, 1)
    torch.cat(inputs_tensor_list, out = inputs_tensor)
    torch.cat(labels_tensor_list, out = labels_tensor)
    inputs_tensor = inputs_tensor.view(batch_size, 1237)
    
    return inputs_tensor, labels_tensor

In [25]:
def computeF1(model, inputs, labels):
    '''
    This function computes the F1 score of the model.
    
    Args:
        model: The model which is being used to compute the F1 score.
        inputs: The inputs to the model.
        labels: The expected outputs from the model.
    
    Returns:
        tp(int):        True Positives.
        fp(int):        False Positives.
        fn(int):        False Negaitves.
        precision(int): The Precision of the F1 score.
        recall(int):    The recall of the F1 score. 
        f1(int):        The computed F1 score.
        
    '''
    tp = 0
    fp = 0
    fn = 0
    for j in range(0, len(inputs), BATCH_SIZE):
        inputs, labels = dataLoader(j, BATCH_SIZE, input_, output_, INPUT_DIM)
        inputs = Variable(inputs)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        for l, p in zip(labels, predicted):
            if l == 1 and p == 1:
                tp += 1
            if l == 0 and p == 1:
                fp += 1
            if l == 1 and p == 0:
                fn += 1
    
    if tp == 0 and fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)
    if tp == 0 and fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)
    if precision == 0 and recall == 0:
        f1 = 0
    else:
        f1 = (2 * precision * recall) / (precision + recall)
        
    return tp, fp, fn, precision, recall, f1

In [26]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(FFNN, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()

        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()

        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.relu3 = nn.ReLU()

        self.fc4 = nn.Linear(hidden_dim2, output_dim)
    
    def forward(self, x):
        #x = x.view(-1, 1337)
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        
        return out

In [32]:
model = FFNN(INPUT_DIM, HIDDEN_DIM1, HIDDEN_DIM2, HIDDEN_DIM3, OUTPUT_DIM)

In [33]:
LEARNING_RATE = 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE)

# Loss Function
criterion = nn.CrossEntropyLoss()

In [34]:
iterations = 0
for epoch in range (1):
    for t, l in zip(list_of_train_files, list_of_label_files):
        input_ = np.load(t)
        output_ = np.load(l)
        for i in range(0, len(input_), BATCH_SIZE):
            inputs, labels = dataLoader(i, BATCH_SIZE, input_, output_, INPUT_DIM)
            inputs , labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = model(inputs)   
            loss = criterion(outputs, labels)        
            loss.backward()
            optimizer.step()
            iterations += 1

            if iterations % 500 == 0:
                tp, fp, fn, precision, recall, f1 = computeF1(model, input_dev, output_dev)
                print ('Epoch: ' + str(epoch))
                print ('Iteration: ' + str(iterations))
                print ('True Positives: ' + str(tp))
                print ('False Positives: ' + str(fp))
                print ('False Negatives: ' + str(fn))
                print ('F1: ' + str(f1))


IndexError: index 27474 is out of bounds for axis 0 with size 27474

In [20]:
for i in range(0, 5, 2):
    print (i)

0
2
4
